In [ ]:
!ngrok config add-authtoken 1X9LyYMW3fNLTb7iQSsYYvguttE_NAhptrGLdA1Hdon1cZPa

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

In [ ]:
!ngrok config add-authtoken 1X9LyYMW3fNLTb7iQSsYYvguttE_NAhptrGLdA1Hdon1cZPa

In [ ]:
import tensorflow as tf
import pickle
from google.colab import drive

# #Mount Google Drive if not already mounted
# drive.mount('/content/drive')

# # Define the directory where the files are saved
# save_dir = "/content/drive/MyDrive/models/"

# #Load the LSTM model (you can use the legacy h5 file format or convert to the new keras format)
# lstm_model = tf.keras.models.load_model(save_dir + "lstm_yield_model.h5")
# print("LSTM model loaded successfully.")

# # Load the feature scaler (scaler_X)
# with open(save_dir + "scaler_X.pkl", "rb") as f:
#     scaler_X = pickle.load(f)
# print("Feature scaler loaded successfully.")

# # Load the target scaler (scaler_y)
# with open(save_dir + "scaler_y.pkl", "rb") as f:
#     scaler_y = pickle.load(f)
# print("Target scaler loaded successfully.")


In [ ]:
1# Import necessary libraries
import os
import time
import pickle
import numpy as np
import pandas as pd
import nest_asyncio
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import tensorflow as tf

# Apply nest_asyncio to allow using asyncio in notebook environments
nest_asyncio.apply()

# -----------------------------
# FastAPI App Setup
# -----------------------------
app = FastAPI(
    title="Agriculture AI Models API",
    description="API for agricultural prediction models"
)

# Enable CORS middleware (allowing requests from any origin)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# -----------------------------
# Define Input Models
# -----------------------------
# class YieldPredictionInput(BaseModel):
#     latitude: float
#     longitude: float
#     crop_type: str
#     target_year: int  # The year for which yield is to be predicted

# # -----------------------------
# Load Pre-Trained LSTM Model and Scalers from Google Drive
# -----------------------------
# from google.colab import drive

# # Mount Google Drive if not already mounted
# drive.mount('/content/drive', force_remount=True)

# # Define the correct path for model and scaler files in Google Drive
# save_dir = "/content/drive/MyDrive/models/"  # Update if your files are in another folder

# # try:
#    lstm_model = tf.keras.models.load_model(save_dir + "lstm_yield_model.h5")
#    with open(save_dir + "scaler_X.pkl", "rb") as f:
#        scaler_X = pickle.load(f)
#    with open(save_dir + "scaler_y.pkl", "rb") as f:
#        scaler_y = pickle.load(f)

#     print("✅ LSTM model and scalers loaded successfully from Google Drive.")
# #except Exception as e:
#     print("❌ Error loading LSTM model or scalers:", str())
#     lstm_model = None

# -----------------------------
# Dummy Function to Retrieve Past Features
# -----------------------------
def get_past_year_features(latitude, longitude, crop_type, target_year, window_size=3):
    """
    Dummy function to simulate retrieval of the past 'window_size' years
    of remote sensing features (e.g., NDVI, EVI, LSWI) for the given location and crop.

    In a real application, this function would query a database or use an API
    to fetch historical data based on the provided coordinates and crop type.

    For demonstration, we return a constant array.
    """
    # Assume we have 3 features: NDVI, EVI, LSWI
    n_features = 3
    # Create a dummy sequence (for example, use constant values)
    # Here, we assume that for the previous window_size years, the features are as follows:
    dummy_sequence = np.array([[0.65, 0.45, 0.20]] * window_size)  # shape: (window_size, n_features)
    return dummy_sequence

# -----------------------------
# Define API Endpoints
# -----------------------------
@app.get("/")
def read_root():
    return {
        "message": "Welcome to Agriculture AI Models API",
        "available_endpoints": ["/predict/crop-yield", "/predict/water-requirement", "/predict/soil-analysis", "/predict/disease-detection"]
    }

@app.get("/health")
def health_check():
    """Health check endpoint to verify the API is running."""
    return {"status": "healthy", "timestamp": time.time()}

# @app.post("/predict/crop-yield")
# def predict_crop_yield(input_data: YieldPredictionInput):
#     """
#     Predicts crop yield using the pre-trained LSTM time-series model.

#     Input:
#       - latitude, longitude: Coordinates of the field.
#       - crop_type: The crop for which yield is to be predicted.
#       - target_year: The year for which prediction is required.

#     Process:
#       1. Retrieve the past years’ features using a dummy function (or database/API in production).
#       2. Normalize the feature sequence using scaler_X.
#       3. Reshape the sequence to match the LSTM input shape.
#       4. Use the LSTM model to predict the scaled yield.
#       5. Inverse-transform the prediction to get the yield in original units.

#     Returns:
#       - The predicted yield in tons per hectare.
#     """
#     if lstm_model is None:
#         raise HTTPException(status_code=500, detail="Yield prediction model is not available.")

#     # Step 1: Retrieve past sequence of features for the given coordinates and crop.
#     window_size = 3  # As used in model training
#     sequence = get_past_year_features(
#         latitude=input_data.latitude,
#         longitude=input_data.longitude,
#         crop_type=input_data.crop_type,
#         target_year=input_data.target_year,
#         window_size=window_size
#     )
#     # Debug print: shape of retrieved sequence
#     print("Retrieved sequence shape:", sequence.shape)

#     # Step 2: Normalize the sequence using the loaded scaler_X.
#     # Reshape sequence to 2D array for scaling then back to 3D.
#     n_features = sequence.shape[1]
#     sequence_flat = sequence.reshape(-1, n_features)
#     sequence_scaled_flat = scaler_X.transform(sequence_flat)
#     sequence_scaled = sequence_scaled_flat.reshape(1, window_size, n_features)  # LSTM expects shape (1, window_size, n_features)

#     # Step 3: Predict yield using the LSTM model.
#     y_pred_scaled = lstm_model.predict(sequence_scaled)
#     y_pred = scaler_y.inverse_transform(y_pred_scaled)

#     # Return prediction as response.
#     return {
#         "crop_type": input_data.crop_type,
#         "target_year": input_data.target_year,
#         "latitude": input_data.latitude,
#         "longitude": input_data.longitude,
#         "predicted_yield": float(y_pred[0][0]),
#         "unit": "tons per hectare"
#     }

# -----------------------------
# Function to Start the Server (for Notebook/Local Use)
# -----------------------------
def start_server():
    # Start ngrok tunnel for external access (useful for Colab or local testing)
    ngrok_tunnel = ngrok.connect(8000)
    print(f"Public URL: {ngrok_tunnel.public_url}")
    print(f"API docs available at: {ngrok_tunnel.public_url}/docs")

    # Run the FastAPI server
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Uncomment the line below to start the server if running as main module
# if __name__ == "__main__":
#  start_server()


In [ ]:
from fastapi.responses import JSONResponse

# ***FastAPI Integration***

In [ ]:
class CHSPredictionInput(BaseModel):
    end_year: int
    start_year: int
    x_min: float
    y_min: float
    x_max: float
    y_max: float

In [ ]:
@app.post("/predict/ndvi")
def predict_ndvi(input_data: CHSPredictionInput):
    try:
        df = create_ndvi_dataset(input_data.start_year, input_data.end_year, input_data.x_min, input_data.y_min, input_data.x_max, input_data.y_max)
        df = interpolate_ndvi(df)
        df = df.dropna()
        df = df.reset_index(drop=True)
        data = train_and_evaluate(df)
        return JSONResponse(content={"ndvi_values": data})
    except Exception as e:
        print(f"Error in predict_ndvi: {e}")  # Print the error to the console
        raise HTTPException(status_code=500, detail=str(e))  # Return a detailed error response

In [ ]:
@app.post("/predict/ndwi")
def predict_ndwi(input_data: CHSPredictionInput):
  df = create_ndwi_dataset(input_data.start_year, input_data.end_year, input_data.x_min , input_data.y_min , input_data.x_max , input_data.y_max)
  df = interpolate_ndwi(df)
  df = df.dropna()
  df = df.reset_index(drop=True)

  bilstm_train_metrics, bilstm_test_metrics, bilstm_model, bilstm_history = train_and_evaluate(df, model_type='bilstm')

  data = predict_future_ndwi(df, bilstm_model)
  return JSONResponse(content={"ndwi_values": data})

# **NDWI Model**

In [ ]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

ee.Authenticate()
ee.Initialize(project = 'agrisage-454715')

In [ ]:
def generate_full_timestamp_range(start_year, end_year):
    """
    Generate complete timestamp range for wheat growing season
    """
    timestamps = []
    for year in range(start_year, end_year + 1):
        # Wheat growing season from Nov to May
        current_date = datetime(year, 11, 1)
        season_end = datetime(year + 1, 5, 31)

        while current_date <= season_end:
            timestamps.append(current_date)
            current_date += timedelta(days=10)

    return timestamps

def load_ndwi_for_period(start_date, end_date, region):
    """
    Load Sentinel images and calculate NDWI for specific period
    """
    # Convert datetime to ee.Date
    start = ee.Date(start_date.strftime('%Y-%m-%d'))
    end = ee.Date(end_date.strftime('%Y-%m-%d'))

    # Load Sentinel-2 images
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start, end) \
        .filterBounds(region) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

    # If no images, return None
    if collection.size().getInfo() == 0:
        return None

    # Calculate NDWI composite
    ndwi_composite = collection \
        .map(lambda image: image.normalizedDifference(['B3', 'B8']).rename('NDWI')) \
        .mean()

    # Extract mean NDWI value
    try:
        mean_ndwi = ndwi_composite.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=10,
            maxPixels=1e9
        ).get('NDWI').getInfo()
        return mean_ndwi
    except:
        return None

def create_ndwi_dataset(start_year, end_year, x_min , y_min, x_max , y_max ):
    """
    Create comprehensive NDWI dataset
    """
  # Define region of interest (example: Punjab, India)
    xmin, ymin = x_min, y_min
    xmax, ymax = x_max, y_max
    punjab = ee.Geometry.Rectangle([x_min, y_min,x_max, y_max])

    # Generate full timestamp range
    timestamps = generate_full_timestamp_range(start_year, end_year)

    # Prepare data collection
    data = []
    for timestamp in timestamps:
        # 10-day period
        period_end = timestamp + timedelta(days=10)

        # Calculate NDWI
        ndwi = load_ndwi_for_period(timestamp, period_end, punjab)

        data.append({
            'timestamp': timestamp,
            'NDWIi': ndwi
        })

    # Convert to DataFrame
    df = pd.DataFrame(data)
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month

    return df

In [ ]:
def interpolate_ndwi(df):
    """
    Interpolate NDWI values with improved seasonal handling and edge cases
    """
    # Create a copy to avoid modifying original data
    df_filtered = df.copy()

    # Convert timestamp to datetime if it isn't already
    df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])

    # Create seasonal features
    df_filtered['month'] = df_filtered['timestamp'].dt.month
    df_filtered['day'] = df_filtered['timestamp'].dt.day
    df_filtered['day_of_year'] = df_filtered['timestamp'].dt.dayofyear

    # Calculate seasonal medians for each day-month combination
    seasonal_medians = df_filtered.groupby(['month', 'day'])['NDWIi'].transform('median')

    # First pass: Fill missing values with seasonal medians
    df_filtered['NDWIi_interpolated'] = df_filtered['NDWIi'].fillna(seasonal_medians)

    # Second pass: Apply linear interpolation within each year
    for year in df_filtered['year'].unique():
        year_mask = df_filtered['year'] == year
        df_filtered.loc[year_mask, 'NDWIi_interpolated'] = (
            df_filtered.loc[year_mask, 'NDWIi_interpolated']
            .interpolate(method='linear', limit_direction='forward', limit=30)  # Limit to 30 days forward
        )

    # Third pass: Fill any remaining NaN with seasonal patterns
    remaining_nans = df_filtered['NDWIi_interpolated'].isna()
    if remaining_nans.any():
        # Calculate the average NDWI pattern across all years
        seasonal_pattern = (
            df_filtered.groupby('day_of_year')['NDWIi']
            .mean()
            .interpolate(method='cubic')  # Smooth the seasonal pattern
        )

        # Fill remaining NaNs with the seasonal pattern
        for idx in df_filtered[remaining_nans].index:
            day_of_year = df_filtered.loc[idx, 'day_of_year']
            df_filtered.loc[idx, 'NDWIi_interpolated'] = seasonal_pattern[day_of_year]

        return df_filtered

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
import matplotlib.pyplot as plt

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

def prepare_data(df, seq_length=6):
    train_data = df[df['year'] < 2024]['NDWIi_interpolated'].values
    test_data = df[df['year'] >= 2024]['NDWIi_interpolated'].values

    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data.reshape(-1, 1))
    test_scaled = scaler.transform(test_data.reshape(-1, 1))

    X_train, y_train = create_sequences(train_scaled, seq_length)
    X_test, y_test = create_sequences(test_scaled, seq_length)

    return X_train, y_train, X_test, y_test, scaler

def build_bilstm_model(seq_length):
    model = Sequential([
        Bidirectional(LSTM(64, activation='relu', return_sequences=True),
                     input_shape=(seq_length, 1)),
        Dropout(0.2),
        Bidirectional(LSTM(32, activation='relu')),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')
    return model

def evaluate_predictions(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    accuracy = 1 - (rmse / (np.max(y_true) - np.min(y_true)))

    return {
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Accuracy': accuracy
    }

def train_and_evaluate(df, model_type='bilstm'):
    # Prepare data
    X_train, y_train, X_test, y_test, scaler = prepare_data(df)

    # Select model type
    if model_type.lower() == 'bilstm':
        model = build_bilstm_model(X_train.shape[1])
        model_name = 'Bidirectional LSTM'
    else:
        raise ValueError("Model type must be 'bilstm'")

    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )

    # Make predictions
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)

    # Inverse transform predictions
    train_pred = scaler.inverse_transform(train_pred)
    test_pred = scaler.inverse_transform(test_pred)
    y_train_orig = scaler.inverse_transform(y_train)
    y_test_orig = scaler.inverse_transform(y_test)

    # Evaluate model
    train_metrics = evaluate_predictions(y_train_orig, train_pred)
    test_metrics = evaluate_predictions(y_test_orig, test_pred)

    return train_metrics, test_metrics, model, history

In [ ]:
def forecast_future(model, last_sequence, scaler, n_days=10):
    """
    Forecast future NDWI values using the trained Bi-LSTM model

    Parameters:
    model: trained Bi-LSTM model
    last_sequence: last known sequence of NDWI values
    scaler: fitted MinMaxScaler
    n_days: number of days to forecast
    """
    predictions = []
    current_sequence = last_sequence.copy()

    # Make predictions for n_days
    for _ in range(n_days):
        # Reshape sequence for model input
        current_sequence_scaled = scaler.transform(current_sequence.reshape(-1, 1))
        X = current_sequence_scaled.reshape(1, len(current_sequence), 1)

        # Predict next value
        next_pred_scaled = model.predict(X, verbose=0)
        next_pred = scaler.inverse_transform(next_pred_scaled)[0][0]

        # Append prediction
        predictions.append(next_pred)

        # Update sequence for next prediction
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = next_pred

    return np.array(predictions)

In [ ]:
def predict_future_ndwi(df, trained_model, seq_length=6, forecast_days=1):
    """
    Main function to prepare data and make future predictions
    """
    # Prepare the last known sequence
    last_known_values = df['NDWIi_interpolated'].values[-seq_length:]

    # Get the scaler
    all_values = df['NDWIi_interpolated'].values
    scaler = MinMaxScaler()
    scaler.fit(all_values.reshape(-1, 1))

    # Make predictions
    future_predictions = forecast_future(trained_model, last_known_values,
                                       scaler, forecast_days)

    # Create dates for predictions
    last_date = pd.to_datetime(df['timestamp'].iloc[-1])
    future_dates = [last_date + pd.Timedelta(days=i+1) for i in range(forecast_days)]

    # Create forecast DataFrame
    forecast_df = pd.DataFrame({
        'Date': future_dates,
        'Predicted_NDWI': future_predictions
    })

    return forecast_df

# **NDVI MODEL**

In [ ]:

import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

ee.Authenticate()
ee.Initialize(project = 'agrisage-454715')

def generate_full_timestamp_range(start_year, end_year):
    """
    Generate complete timestamp range for wheat growing season
    """
    timestamps = []
    for year in range(start_year, end_year + 1):
        # Wheat growing season from Nov to May
        current_date = datetime(year, 11, 1)
        season_end = datetime(year + 1, 5, 31)

        while current_date <= season_end:
            timestamps.append(current_date)
            current_date += timedelta(days=10)

    return timestamps

def load_ndvi_for_period(start_date, end_date, region):

    # Convert datetime to ee.Date
    start = ee.Date(start_date.strftime('%Y-%m-%d'))
    end = ee.Date(end_date.strftime('%Y-%m-%d'))

    # Load Sentinel-2 images
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start, end) \
        .filterBounds(region) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

    # If no images, return None
    if collection.size().getInfo() == 0:
        return None

    # Calculate NDVI composite
    ndvi_composite = collection \
        .map(lambda image: image.normalizedDifference(['B8', 'B4']).rename('NDVI')) \
        .mean()

    # Extract mean NDVI value
    try:
        mean_ndvi = ndvi_composite.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=10,
            maxPixels=1e9
        ).get('NDVI').getInfo()
        return mean_ndvi
    except:
        return None

def create_ndvi_dataset(start_year, end_year, x_min , y_min, x_max , y_max ):
    """
    Create comprehensive NDVI dataset
    """
  # Define region of interest (example: Punjab, India)
    xmin, ymin = x_min, y_min
    xmax, ymax = x_max, y_max
    punjab = ee.Geometry.Rectangle([x_min, y_min,x_max, y_max])


    # Generate full timestamp range
    timestamps = generate_full_timestamp_range(start_year, end_year)

    # Prepare data collection
    data = []
    for timestamp in timestamps:
        # 10-day period
        period_end = timestamp + timedelta(days=10)

        # Calculate NDVI
        ndvi = load_ndvi_for_period(timestamp, period_end, punjab)

        data.append({
            'timestamp': timestamp,
            'NDVIi': ndvi
        })

    # Convert to DataFrame
    df = pd.DataFrame(data)
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month

    return df

In [ ]:
def interpolate_ndvi(df):

    # Create a copy to avoid modifying original data
    df_filtered = df.copy()

    # Convert timestamp to datetime if it isn't already
    df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])

    # Create seasonal features
    df_filtered['month'] = df_filtered['timestamp'].dt.month
    df_filtered['day'] = df_filtered['timestamp'].dt.day
    df_filtered['day_of_year'] = df_filtered['timestamp'].dt.dayofyear

    # Calculate seasonal medians for each day-month combination
    seasonal_medians = df_filtered.groupby(['month', 'day'])['NDVIi'].transform('median')

    # First pass: Fill missing values with seasonal medians
    df_filtered['NDVIi_interpolated'] = df_filtered['NDVIi'].fillna(seasonal_medians)

    # Second pass: Apply linear interpolation within each year
    for year in df_filtered['year'].unique():
        year_mask = df_filtered['year'] == year
        df_filtered.loc[year_mask, 'NDVIi_interpolated'] = (
            df_filtered.loc[year_mask, 'NDVIi_interpolated']
            .interpolate(method='linear', limit_direction='forward', limit=30)  # Limit to 30 days forward
        )

    # Third pass: Fill any remaining NaN with seasonal patterns
    remaining_nans = df_filtered['NDVIi_interpolated'].isna()
    if remaining_nans.any():
        # Calculate the average NDVI pattern across all years
        seasonal_pattern = (
            df_filtered.groupby('day_of_year')['NDVIi']
            .mean()
            .interpolate(method='cubic')  # Smooth the seasonal pattern
        )

        # Fill remaining NaNs with the seasonal pattern
        for idx in df_filtered[remaining_nans].index:
            day_of_year = df_filtered.loc[idx, 'day_of_year']
            df_filtered.loc[idx, 'NDVIi_interpolated'] = seasonal_pattern[day_of_year]

    return df_filtered

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
from datetime import timedelta

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length)])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

def prepare_data(df, seq_length=6):
    train_data = df[df['year'] < 2024]['NDVIi_interpolated'].values
    test_data = df[df['year'] >= 2024]['NDVIi_interpolated'].values
    test_dates = df[df['year'] >= 2024]['timestamp'].values  # Get test set dates

    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data.reshape(-1, 1))
    test_scaled = scaler.transform(test_data.reshape(-1, 1))

    X_train, y_train = create_sequences(train_scaled, seq_length)
    X_test, y_test = create_sequences(test_scaled, seq_length)

    return X_train, y_train, X_test, y_test, scaler, test_dates[-1]

def build_lstm_model(seq_length):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=(seq_length, 1), return_sequences=True),
        Dropout(0.2),
        LSTM(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')
    return model

def evaluate_predictions(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    # Calculate accuracy as 1 - normalized RMSE
    accuracy = 1 - (rmse / (np.max(y_true) - np.min(y_true)))

    return {
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Accuracy': accuracy
    }

# Predict next 10 days NDVI

def predict_next_10_days(model, last_sequence, scaler, last_date):
    predictions = []
    dates = []
    input_seq = last_sequence.copy()

    for i in range(1):  # Predict for next 10 days which will be calculted to give as a new predicted ndvi composite
        pred = model.predict(input_seq.reshape(1, -1, 1))
        pred_inv = scaler.inverse_transform(pred)
        predictions.append(pred_inv[0][0])

        # Convert last_date to datetime if needed
        last_date = pd.to_datetime(last_date)
        dates.append(last_date + timedelta(days=i + 1))  # Generate future timestamps

        input_seq = np.append(input_seq[1:], pred)

    return predictions, dates


# Main execution
def train_and_evaluate(df):
    # Prepare data
    X_train, y_train, X_test, y_test, scaler,last_test_date = prepare_data(df)

    # Build and train model
    model = build_lstm_model(X_train.shape[1])
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )

    # Make predictions
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)

    # Inverse transform predictions
    train_pred = scaler.inverse_transform(train_pred)
    test_pred = scaler.inverse_transform(test_pred)
    y_train_orig = scaler.inverse_transform(y_train)
    y_test_orig = scaler.inverse_transform(y_test)

    # Evaluate model
    train_metrics = evaluate_predictions(y_train_orig, train_pred)
    test_metrics = evaluate_predictions(y_test_orig, test_pred)

    # Predict next 10 days NDVI
    last_sequence = X_test[-1]  # Get last sequence from test data
    next_10_days_predictions, next_10_days_dates = predict_next_10_days(model, last_sequence, scaler, last_test_date)

    global global_next_10_days_prediction
    global_next_10_days_prediction = list(next_10_days_predictions)

    return global_next_10_days_prediction

In [ ]:
def convert(df):
  new_df = df.copy()



In [ ]:
start_server()

Public URL: https://615d-34-125-101-193.ngrok-free.app
API docs available at: https://615d-34-125-101-193.ngrok-free.app/docs


INFO:     Started server process [465]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2409:40e3:2093:753a:a9c5:d501:107:3738:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2409:40e3:2093:753a:a9c5:d501:107:3738:0 - "GET /openapi.json HTTP/1.1" 200 OK
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.4465 - val_loss: 0.1102
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.4343 - val_loss: 0.1069
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 0.4246 - val_loss: 0.1040
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.4180 - val_loss: 0.1010
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.4087 - val_loss: 0.0981
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - loss: 0.4006 - val_loss: 0.0952
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.3900 - val_loss: 0.0923
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.3798 - val_loss: 0.0894
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 0.3751 - val_loss: 0.0864
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.3630 - val_loss: 0.0834
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.3538 - val_loss: 0.0803
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.3453 - val_loss: 0.0773
E

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 528ms/step

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 919ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
INFO:     2409:40e3:2093:753a:a9c5:d501:107:3738:0 - "POST /predict/ndvi HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py",